In [155]:
#main
import pandas as pd
import numpy as np

#plot
from matplotlib import pyplot as plt
import seaborn as sns

#dop
import re
from tqdm import tqdm 
from IPython.display import display


# option
%matplotlib inline
pd.set_option("display.max_columns", 300)
plt.rcParams['figure.figsize'] = (15, 10)


PATH_TRAIN = 'data/train.csv'
PATH_TEST = 'data/test.csv'
TARGET_COL = 'SalePrice'

In [122]:
# Загружаем данные выделяем 'Y'

train = pd.read_csv(PATH_TRAIN)
test = pd.read_csv(PATH_TEST)
y_train_main = train[TARGET_COL]
df = pd.concat([train.drop(TARGET_COL, axis=1), test], ignore_index=True)

In [123]:
# создание таблицы с пропусками

def nan_info(df):
    global null_view
    try:
        null_view = df[[col for col in df.columns if df[col].isna().sum() > 0]].isna().sum().sort_values(ascending = True)
        null_view = pd.DataFrame(null_view, columns=['NANs'])
        null_view['PERCENT'] = null_view.NANs.apply(lambda x: round((x/len(df))*100, 2))
        null_view['TYPE'] = df.dtypes
    except:
        print('---[ ERROR ]---')
        return null_view
    return null_view

NaN_info(df)

,NANs,PERCENT,TYPE
TotalBsmtSF,1,0.03,float64
GarageArea,1,0.03,float64
GarageCars,1,0.03,float64
KitchenQual,1,0.03,object
Electrical,1,0.03,object
BsmtUnfSF,1,0.03,float64
BsmtFinSF2,1,0.03,float64
BsmtFinSF1,1,0.03,float64
SaleType,1,0.03,object
Exterior1st,1,0.03,object


In [124]:
col_nan_how_no = ['Alley','BsmtQual', 'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
                  'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond',
                  'PoolQC','Fence','MiscFeature']

def nan_how_no(df, col_list):
    for col in col_list:
        df[col].replace(np.nan, 'No', inplace=True)
        
        
nan_how_no(df, col_nan_how_no)
nan_info(df)

,NANs,PERCENT,TYPE
BsmtUnfSF,1,0.03,float64
GarageCars,1,0.03,float64
KitchenQual,1,0.03,object
Electrical,1,0.03,object
TotalBsmtSF,1,0.03,float64
GarageArea,1,0.03,float64
BsmtFinSF2,1,0.03,float64
SaleType,1,0.03,object
Exterior2nd,1,0.03,object
Exterior1st,1,0.03,object


In [185]:
def encoding(x, y, LE):
    for col in x.columns:
        if x[col].dtype == 'object':
            x[col].replace(np.nan, '0', inplace=True)
            x.loc[:, col] = LE.fit_transform(x.loc[:, col])
    y.loc[~y.isna()] = LE.fit_transform(y.loc[~y.isna()])

    return LE

def get_variable(df, col_pred):
    # Делит данные по индексу которые нужно предсказать
    y_now = df[col_pred].copy()
    x_now = df[set(df.columns) - set(col_pred)].copy()
    
    LE = encoding(x_now, y_now, LabelEncoder())
    
    index_train = y_now[~y_now.isna()].index
    index_target = y_now[y_now.isna()].index
    
    x_train = x_now.drop(index=index_target)
    y_train = y_now.drop(index=index_target)
    x_now.drop(index=index_train, inplace=True)
    y_now.drop(index=index_train, inplace=True)
    return x_train, y_train.astype('int32'), x_now, y_now, LE

def predict_regressor(x_train, y_train, x_target):
    lgbm = lgb.LGBMRegressor()
    lgbm.fit(x_train, y_train)
    pred_miss = lgbm.predict(x_target)
    print('Start: predict_regressor')
    print('-------------------------------')
    return pred_miss


def predict_classifier(x_train, y_train, x_target):
    lgbm = lgb.LGBMClassifier()
    lgbm.fit(x_train, y_train)
    pred_miss = lgbm.predict(x_target)
    print('Start: predict_classifier')
    return pred_miss

def nan_predict(df_start):
    # Возращает новый dataframe с заполнеными NaN 
    import lightgbm as lgb
    from sklearn.preprocessing import LabelEncoder
    
    df = df_start.copy()
    nan_info(df)
    
    predicted_features = null_view.index
    
    # Разделение на датафрейм
    for col_pred in predicted_features: 
        print(f"feature: {col_pred}, missing values: {null_view['PERCENT'][col_pred]}%, type: {null_view['TYPE'][col_pred]}")
        
        x_train, y_train, x_target, y_target, LE = get_variable(df, col_pred)
        
        if df[col_pred].dtype == 'object' or df[col_pred].dtype == 'bool':
            y_pred = predict_classifier(x_train, y_train, x_target)
            y_pred = LE.inverse_transform(y_pred)
        else:
            y_pred = predict_regressor(x_train, y_train, x_target)
        
        df.loc[y_target.index, y_target.name] = y_pred
        print(f"The first 100 predicted missing values: \n{y_pred[:100]}")
        print('-------------------------------\n')
        
        
    return df

feature: BsmtUnfSF, missing values: 0.03%, type: float64
Start: predict_regressor
-------------------------------
The first 100 predicted missing values: 
[0.16147999]
-------------------------------

feature: GarageCars, missing values: 0.03%, type: float64
Start: predict_regressor
-------------------------------
The first 100 predicted missing values: 
[-0.00126173]
-------------------------------

feature: KitchenQual, missing values: 0.03%, type: object
Start: predict_classifier
The first 100 predicted missing values: 
['Ex']
-------------------------------

feature: Electrical, missing values: 0.03%, type: object
Start: predict_classifier
The first 100 predicted missing values: 
['FuseA']
-------------------------------

feature: TotalBsmtSF, missing values: 0.03%, type: float64
Start: predict_regressor
-------------------------------
The first 100 predicted missing values: 
[0.34537115]
-------------------------------

feature: GarageArea, missing values: 0.03%, type: float64
Sta

---

In [182]:
y_target.name

'BsmtUnfSF'